<a href="https://colab.research.google.com/github/PotatoNinja14/102103165-SESS_LE1/blob/main/about-lab-eval/102103165_UmangSrivastava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: **Umang Srivastava**  
Email: `usrivastava1_be21@thapar.edu`  
Roll No: **102103165**  
Group: **4CO6**  
Start Timestamp: 20240911-1000  

## Question

Given the dimensionality d of inputs, a sequence H of channel size for each hidden layer, and number of classes C, define a function to

Take as argument a vector of raw inputs x;
Define a neural network classifier with d input channels, len(H) hidden layers, each subsequent layer bearing [h1,h2,...] channels finally resulting in C logits corresponding to each class;
Activate each intermediate layer with tanh
 activation; and
Return the logits.
The function shall be tested for consistency, correctness and efficiency as applicable.

Use PyTorch/Tensorflow for implementation. Use of internet resources is dicouraged in the interest of time, though not prohibited.

## Solution

In [4]:
import torch
import torch.nn as nn

def createNnClassifier(d,H,C) :
  lis = []
  input_layer = nn.Linear(d, H[0])
  lis.append(input_layer)
  lis.append(nn.Tanh())

  for i in range(1, len(H)):
      hlis = nn.Linear(H[i-1], H[i])
      lis.append(hlis)
      lis.append(nn.Tanh())

  llis = nn.Linear(H[-1], C)
  lis.append(llis)

  model = nn.Sequential(*lis)

  return model

## Test Case

In [5]:
d,H,C = 4,[3,8],1
N = 8  # batch_size
x = torch.rand(N,d)

model = createNnClassifier(d,H,C)
model

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): Tanh()
  (2): Linear(in_features=3, out_features=8, bias=True)
  (3): Tanh()
  (4): Linear(in_features=8, out_features=1, bias=True)
)

In [6]:
y_hat = torch.argmax(model(x), dim=1)

y_hat.size()

torch.Size([8])